https://bradfieldcs.com/algos/graphs/introduction

###  Introduction to Graphs 

    Vertex
    Edge
    Weight
    Path
    directed graph, or digraph
    Cycle
    directed acyclic graph or a DAG

#### The Graph Abstract Data Type


    Graph() creates a new, empty graph.
    add_vertex(vertex) adds an instance of Vertex to the graph.
    add_edge(from_vertex, to_vertex) Adds a new, directed edge to the graph that connects two vertices.
    add_edge(from_vertex, to_vertex, weight) Adds a new, weighted, directed edge to the graph that connects two vertices.
    get_vertex(key) finds the vertex in the graph named key.
    get_vertices() returns the list of all vertices in the graph.
    in returns True for a statement of the form vertex in graph, if the given vertex is in the graph, False otherwise.


    Beginning with the formal definition for a graph there are several ways we can implement the graph ADT in Python. We will see that there are trade-offs in using different representations to implement the ADT described above. There are two well-known implementations of a graph, the adjacency matrix and the adjacency list. We will explain both of these options, and then implement one as a Python class.

### Representing a Graph 

#### The Adjacency Matrix

    One of the easiest ways to implement a graph is to use a two-dimensional matrix. In this matrix implementation, each of the rows and columns represent a vertex in the graph. The value that is stored in the cell at the intersection of row v and column w indicates if there is an edge from vertex v to vertex w. When two vertices are connected by an edge, we say that they are adjacent. The diagram below illustrates the adjacency matrix for the example graph we presented earlier. A value in a cell represents the weight of the edge from vertex v to vertex w.
    
    The advantage of the adjacency matrix is that it is simple, and for small graphs it is easy to see which nodes are connected to other nodes. However, notice that most of the cells in the matrix are empty. Because most of the cells are empty we say that this matrix is “sparse.” A matrix is not a very efficient way to store sparse data. In fact, in Python you must go out of your way to even create a matrix structure like the one above.
    
    The adjacency matrix is a good implementation for a graph when the number of edges is large. But what do we mean by large? How many edges would be needed to fill the matrix? Since there is one row and one column for every vertex in the graph, the number of edges required to fill the matrix is |V|^2. A matrix is full when every vertex is connected to every other vertex. There are few real problems that approach this sort of connectivity. The problems we will look at in this section all involve graphs that are sparsely connected.
    

#### The Adjacency List

    A more space-efficient way to implement a sparsely connected graph is to use the unfortunately named adjacency list structure. In an adjacency list implementation we keep a master collection of all the vertices in the Graph object and then each vertex object in the graph maintains a list of the other vertices that it is connected to. In our implementation of the Vertex class we will use a dictionary rather than a list as our master collection, where the dictionary keys are the vertices, and the values are the weights. The diagram below shows an adjacency list representation of the example graph we have been discussing.

#### An Object-Oriented Approach

In [2]:
class Vertex:
    def __init__(self, key):
        self.key = key
        self.neighbors = {}
    
    def add_neighbor(self, neighbor, weight=0):
        self.neighbors[neighbor] = weight
        
    def __str__(self):
        return f"{self.key} neighbors {[x.key for x in self.neighbors]}"
        
    def get_connections(self):
        return self.neighbors.keys()
    
    def get_weight(self, neighbor):
        return self.neighbors[neighbor]

In [3]:
class Graph:
    def __init__(self):
        self.vertices = {}
        
    def add_vertex(self, vertex):
        self.vertices[vertex.key] = vertex
        
    def get_vertex(self, key):
        try:
            return self.vertices[key]
        except KeyError:
            return None
        
    def __contains__(self, key):
        return key in self.vertices
    
    def add_edge(self, from_key, to_key, weight):
        if from_key not in self.vertices:
            self.add_vertex(Vertex(from_key))
        if to_key not in self.vertices:
            self.add_vertex(Vertex(to_key))
            
        self.vertices[from_key].add_neighbor(self.vertices[to_key], weight)
        
    def get_vertices(self):
        return self.vertices.keys()
    
    def __iter__(self):
        return iter(self.vertices.values())

In [4]:
g = Graph()
for i in range(6):
    g.add_vertex(Vertex(i))

In [5]:
g.vertices

{0: <__main__.Vertex at 0x7f92f1e9da60>,
 1: <__main__.Vertex at 0x7f92f1673e20>,
 2: <__main__.Vertex at 0x7f92f1673f40>,
 3: <__main__.Vertex at 0x7f92f1673eb0>,
 4: <__main__.Vertex at 0x7f92f1673f70>,
 5: <__main__.Vertex at 0x7f92f1673df0>}

In [6]:
g.add_edge(0, 1, 5)
g.add_edge(0, 5, 2)
g.add_edge(1, 2, 4)
g.add_edge(2, 3, 9)
g.add_edge(3, 4, 7)
g.add_edge(3, 5, 3)
g.add_edge(4, 0, 1)
g.add_edge(5, 4, 8)
g.add_edge(5, 2, 1)

In [9]:
for v in g:
    for w in v.get_connections():
        print('{} -> {}'.format(v.key, w.key))
    print(v)

0 -> 1
0 -> 5
0 neighbors [1, 5]
1 -> 2
1 neighbors [2]
2 -> 3
2 neighbors [3]
3 -> 4
3 -> 5
3 neighbors [4, 5]
4 -> 0
4 neighbors [0]
5 -> 4
5 -> 2
5 neighbors [4, 2]


#### Using Dictionaries Directly

In [10]:
{
    0: {1: 5, 5: 2},
    1: {2: 4},
    2: {3: 9},
    3: {4: 7, 5: 3},
    4: {0: 1},
    5: {4: 8}
}


{0: {1: 5, 5: 2}, 1: {2: 4}, 2: {3: 9}, 3: {4: 7, 5: 3}, 4: {0: 1}, 5: {4: 8}}

### Word Ladders 

FOOL
POOL
POLL
POLE
PALE
SALE
SAGE

#### Building the Word Ladder Graph

    Word buckets for words that are different by one letter

In [11]:
from collections import defaultdict
from itertools import product
import os

In [149]:
def build_buckets(words):
    buckets = defaultdict(list)
    for word in words:
        for i in range(len(word)):
            bucket = list(word)
            bucket[i] = "_"
            bucket = "".join(bucket)
            
            buckets[bucket].append(word)     
    return buckets     
    
def build_graph(words):
    graph = defaultdict(set)
    buckets = build_buckets(words)
    
    for bucket, mutual_neighbors in buckets.items():
        for word1 in mutual_neighbors:
            for word2 in mutual_neighbors:
                if word1 != word2:
                    graph[word1].add(word2)
                    graph[word2].add(word1)
    return graph
    
def get_words(vocabulary_file):
    for line in open(vocabulary_file, 'r'):
        yield line[:-1]  # remove newline character

In [150]:
vocabulary_file = 'vocabulary.txt'

In [152]:
word_graph = build_graph(get_words(vocabulary_file))

In [153]:
word_graph['FOOL']

{'COOL',
 'FOAL',
 'FOIL',
 'FOOD',
 'FOOT',
 'FOUL',
 'FOWL',
 'POOL',
 'TOOL',
 'WOOL'}

#### Implementing breadth first search

In [154]:
from collections import deque

In [203]:
def traverse(graph, starting_vertex):
    visited = set()
    queue = deque()
    queue.append([starting_vertex])
    while queue:
        popped_item = queue.popleft()
        vertex = popped_item[-1]
        yield vertex, popped_item
        
        for neighbor in graph[vertex]:
            if neighbor not in visited:
                visited.add(neighbor)
                queue.append(popped_item + [neighbor])

In [204]:
def chain_generator(word_graph, from_node, to_node):
    for vertex, path in traverse(word_graph, from_node):
        if vertex == to_node:
            print(' -> '.join(path))

In [205]:
chain_generator(word_graph, "FOOL", "SAGE")

FOOL -> WOOL -> WOOS -> WOGS -> WAGS -> SAGS -> SAGE


#### Breadth First Search Analysis

    Before we continue with other graph algorithms let us analyze the run time performance of the breadth first search algorithm. The first thing to observe is that the while loop is executed, at most, one time for each vertex in the graph ∣V∣. You can see that this is true because a vertex must be white before it can be examined and added to the queue. This gives us O(V) for the while loop. The for loop, which is nested inside the while is executed at most once for each edge in the graph, ∣E∣. The reason is that every vertex is dequeued at most once and we examine an edge from node uuu to node vvv only when node uuu is dequeued. This gives us O(E) for the for loop. combining the two loops gives us O(V+E).

    Of course doing the breadth first search is only part of the task. Following the links from the starting node to the goal node is the other part of the task. The worst case for this would be if the graph was a single long chain. In this case traversing through all of the vertices would be O(V). The normal case is going to be some fraction of ∣V∣ but we would still write O(V).

In [206]:
# time of build_graph?

### A Knight’s Tour 

#### Building the Knight’s Tour Graph

https://stackoverflow.com/questions/20642332/missing-1-required-positional-argument-why

In [58]:
from collections import defaultdict
from functools import cmp_to_key

In [59]:
def add_edge(graph, vertex_a, vertex_b):
    graph[vertex_a].add(vertex_b)
    graph[vertex_b].add(vertex_a)
    

def legal_moves_from(row, col, board_size):
    move_offsets = [(-2,-1),(-1,-2),(1,-2),(-2,1),(-1,2),(2,-1),(1,2),(2,1)]
    for x,y in move_offsets:
        new_row = x+row
        new_col = y+col
        if (new_row in range(0, board_size)) and (new_col in range(0, board_size)):
            yield (new_row, new_col)

def build_graph(board_size):
    graph = defaultdict(set)
    for row in range(board_size):
        for col in range(board_size):
            for to_row, to_col in legal_moves_from(row, col, board_size):
                add_edge(graph, (row, col), (to_row, to_col))
                
    return graph

#### Implementing Knight’s Tour

In [60]:
def traverse(path, current_vertex, total_squares, heuristic, graph):
    if len(path) + 1 == total_squares:
        return path + [current_vertex]

    yet_to_visit = graph[current_vertex] - set(path)
    if not yet_to_visit:
        return False

    if heuristic(graph):
        key = cmp_to_key(heuristic(graph))
    else:
        key = heuristic(graph)
        
    next_vertices = sorted(yet_to_visit, key=key)
    for vertex in next_vertices:
        result = traverse(path + [current_vertex], vertex, total_squares, heuristic, graph)
        if result:
            return result
        


def find_solution_for(board_size, heuristic=lambda graph: None):
    graph = build_graph(board_size)
    total_squares = board_size * board_size
    
    for starting_vertex in graph:
        result = traverse([], starting_vertex, total_squares, heuristic, graph)
        if result:
            return result
        

In [66]:
%time print(find_solution_for(8))

[(0, 0), (1, 2), (0, 4), (1, 6), (2, 4), (0, 3), (1, 1), (2, 3), (0, 2), (1, 0), (2, 2), (0, 1), (1, 3), (0, 5), (1, 7), (2, 5), (0, 6), (1, 4), (2, 6), (0, 7), (1, 5), (2, 7), (3, 5), (4, 3), (3, 1), (5, 0), (4, 2), (2, 1), (3, 3), (4, 1), (2, 0), (3, 2), (4, 0), (6, 1), (5, 3), (3, 4), (4, 6), (6, 7), (7, 5), (5, 4), (7, 3), (6, 5), (7, 7), (5, 6), (3, 7), (4, 5), (5, 7), (3, 6), (4, 4), (6, 3), (7, 1), (5, 2), (6, 0), (7, 2), (6, 4), (7, 6), (5, 5), (4, 7), (6, 6), (7, 4), (6, 2), (7, 0), (5, 1), (3, 0)]
CPU times: user 33.7 s, sys: 0 ns, total: 33.7 s
Wall time: 33.7 s


#### Knight’s Tour Analysis

    There is one last interesting topic regarding the knight’s tour problem, then we will move on to the general version of the depth first search. The topic is performance. In particular, our algorithm is very sensitive to the method you use to select the next vertex to visit. For example, on a five-by-five board you can produce a path in about 1.5 seconds on a reasonably fast computer. But what happens if you try an eight-by-eight board? In this case, depending on the speed of your computer, you may have to wait up to a half hour to get the results! The reason for this is that the knight’s tour problem as we have implemented it so far is an exponential algorithm of size O(k^N), where N is the number of squares on the chess board, and k is a small constant. The diagram below can help us visualize why this is so.

In [62]:
def warnsdorffs_heuristic(graph):

    #Given a graph, return a comparator function that prioritizes nodes
    #with the fewest subsequent moves
    def comparator(a, b):
        return len(graph[a]) - len(graph[b])

    return comparator

In [65]:
%time print(find_solution_for(8, warnsdorffs_heuristic))

[(0, 0), (1, 2), (0, 4), (1, 6), (3, 7), (5, 6), (7, 7), (6, 5), (5, 7), (7, 6), (6, 4), (7, 2), (6, 0), (4, 1), (2, 0), (0, 1), (1, 3), (0, 5), (1, 7), (3, 6), (1, 5), (0, 7), (2, 6), (4, 7), (6, 6), (7, 4), (6, 2), (7, 0), (5, 1), (3, 0), (1, 1), (0, 3), (2, 4), (4, 5), (5, 3), (6, 1), (4, 0), (2, 1), (0, 2), (1, 0), (3, 1), (5, 0), (7, 1), (6, 3), (7, 5), (6, 7), (4, 6), (2, 7), (0, 6), (1, 4), (2, 2), (3, 4), (5, 5), (4, 3), (3, 5), (2, 3), (4, 2), (5, 4), (7, 3), (5, 2), (3, 3), (2, 5), (4, 4), (3, 2)]
CPU times: user 7.64 ms, sys: 0 ns, total: 7.64 ms
Wall time: 7.36 ms


### General Depth First Search 

In [1]:
from collections import defaultdict

In [2]:
simple_graph = {
    'A': ['B', 'D'],
    'B': ['C', 'D'],
    'C': [],
    'D': ['E'],
    'E': ['B', 'F'],
    'F': ['C']
}

In [181]:
def depth_first_search(graph, starting_vertex):
    visited = set()
    counter = 0
    traversal_times = defaultdict(dict)

    traverse(graph, starting_vertex, visited, counter, traversal_times)
    return traversal_times


def traverse(graph, vertex, visited, counter, traversal_times):
    visited.add(vertex)
    counter += 1
    traversal_times[vertex]['discovery'] = counter

    for next_vertex in graph[vertex]:
        if next_vertex not in visited:
            counter = traverse(graph, next_vertex, visited, counter, traversal_times)

    counter += 1
    traversal_times[vertex]['finish'] = counter
    return counter


In [15]:
traversal_times = depth_first_search(simple_graph, 'A')

In [16]:
traversal_times

defaultdict(dict,
            {'A': {'discovery': 1, 'finish': 12},
             'B': {'discovery': 2, 'finish': 11},
             'C': {'discovery': 3, 'finish': 4},
             'D': {'discovery': 5, 'finish': 10},
             'E': {'discovery': 6, 'finish': 9},
             'F': {'discovery': 7, 'finish': 8}})

### Topological Sorting 

    A topological sort takes a directed acyclic graph and produces a linear ordering of all its vertices such that if the graph G contains an edge (v,w) then the vertex v comes before the vertex w in the ordering. 
    Directed acyclic graphs are used in many applications to indicate the precedence of events. Making pancakes is just one example; other examples include software project schedules, precedence charts for optimizing database queries, and multiplying matrices.

    The topological sort is a simple but useful adaptation of a depth first search. The algorithm for the topological sort is as follows:

    1.Perform a depth first search over the graph in order to compute the finish times for each of the vertices.
    2.Store the vertices in a list in decreasing order of finish time.
    3.Return the ordered list as the result of the topological sort


In [274]:
def find_degrees_of_node(graph):
    degrees = {}
    for node in graph:
        degrees[node] = 0
    
    for node, edges in graph.items():
        for destination_node in edges:
            degrees[destination_node] += 1
            
    return degrees

def find_min_degree(graph):
    degrees = find_degrees_of_node(graph)
    return min(degrees)

def sort_dictionary(x, reverse=True):
    return {k: v for k, v in sorted(x.items(), key=lambda item: item[1], reverse=reverse)}
    
def sort_traversal_times(traversal_times):
    for key, value in traversal_times.items():
        traversal_times[key] = value["finish"]
        
    return sort_dictionary(traversal_times)
    
def topological_sort(graph):
    min_node = find_min_degree(graph)
    traversal_times = depth_first_search(simple_graph, min_node)
    sorted_traversal_times = sort_traversal_times(traversal_times)
    return sorted_traversal_times.keys()

In [57]:
topological_sort(simple_graph)

dict_keys(['A', 'B', 'D', 'E', 'F', 'C'])

### Shortest Path with Dijkstra’s Algorithm 

    When you surf the web, send an email, or log in to a laboratory computer from another location on campus a lot of work is going on behind the scenes to get the information on your computer transferred to another computer. The in-depth study of how information flows from one computer to another over the Internet is the primary topic for a class in computer networking. However, we will talk about how the Internet works just enough to understand another very important graph algorithm.

#### Dijkstra’s Algorithm

In [58]:
import heapq

In [145]:
def calculate_distances(graph, starting_vertex):
    distances = {vertex: [float("infinity"), None] for vertex in graph}
    distances[starting_vertex] = [0, None]
    
    pq = [(0, starting_vertex)]
    
    while len(pq)>0:
        # print("before:", pq)
        current_distance, current_vertex = heapq.heappop(pq)
        # print("after:", pq)
        if current_distance > distances[current_vertex][0]:
            continue
        
        for neighbor, weight in graph[current_vertex].items():
            # print(current_vertex, neighbor, weight)
            distance = current_distance + weight
                
            if distance < distances[neighbor][0]:
                # print("less", neighbor)
                # print("hit", pq)
                distances_object = distances[neighbor]
                distances_object[0] = distance
                distances_object[1] = current_vertex
                
                heapq.heappush(pq, (distance, neighbor))

    return distances

In [152]:
example_graph = {
    'U': {'V': 2, 'W': 5, 'X': 1},
    'V': {'U': 2, 'X': 2, 'W': 3},
    'W': {'V': 3, 'U': 5, 'X': 3, 'Y': 1, 'Z': 5},
    'X': {'U': 1, 'V': 2, 'W': 3, 'Y': 1},
    'Y': {'X': 1, 'W': 1, 'Z': 1},
    'Z': {'W': 5, 'Y': 1},
}

In [149]:
print(calculate_distances(example_graph, 'X'))
# => {'U': 1, 'W': 2, 'V': 2, 'Y': 1, 'X': 0, 'Z': 2}

{'U': [1, 'X'], 'V': [2, 'X'], 'W': [2, 'Y'], 'X': [0, None], 'Y': [1, 'X'], 'Z': [2, 'Y']}


In [150]:
print(calculate_distances(example_graph, 'U'))

{'U': [0, None], 'V': [2, 'U'], 'W': [3, 'Y'], 'X': [1, 'U'], 'Y': [2, 'X'], 'Z': [3, 'Y']}


#### Analysis of Dijkstra’s Algorithm

    We will now consider the running time of Dijkstra’s algorithm.

    Building the distances dictionary takes O(V) time since we add every vertex in the graph to the dictionary.

    The while loop is executed once for every entry that gets added to the priority queue. An entry can only be added when we explore an edge, so there are at most O(E) iterations of the while loop.

    The for loop is executed at most once for every vertex, since the current_distance > distances[current_vertex] check ensures that we only process a vertex once. The for loop iterates over outgoing edges, so among all iterations of the while loop, the body of the for loop executes at most O(E) times.

    Finally, if we consider that each priority queue operation (adding or removing an entry) is O(logE), we conclude that the total running time is O(V+ElogE).

### Strongly Connected Components 

https://stackoverflow.com/questions/11277432/how-can-i-remove-a-key-from-a-python-dictionary

    We formally define a strongly connected component, C, of a graph G, as the largest subset of vertices C⊂V such that for every pair of vertices v,w ∈ C, we have a path from v to w and a path from w to v. The illustration below shows a simple graph with three strongly connected components. The strongly connected components are identified by the different shaded areas.
    
    Once the strongly connected components have been identified we can show a simplified view of the graph by combining all the vertices in one strongly connected component into a single larger vertex. The simplified version of the graph above is shown below.

    The transpose of a graph G is defined as the graph G^T where all the edges in the graph have been reversed. That is, if there is a directed edge from node A to node B in the original graph then G^T will contain an edge from node B to node A. The diagrams below show a simple graph and its transposition.

    We can now describe the algorithm to compute the strongly connected components for a graph.

    1.Perform a depth first search for the graph G to compute the finish times for each vertex.
    
    2.Compute G^T
    
    3.Perform a depth first search for the graph G^T but in the main loop explore each vertex in decreasing order of finish time.
    
    4.Each tree in the forest computed in step 3 is a strongly connected component. Output the vertex ids for each vertex in each tree in the forest to identify the component.


In [213]:
from collections import defaultdict

In [289]:
scc_graph = {
    'A': {'B':1, },
    'B': {'E':1, 'C':1, },
    'C': {'C':1, 'F':1, },
    'D': {'B':1, 'G':1, },
    'E': {'D':1, 'A':1,},
    'F': {'H':1, },
    'G': {'E':1, },
    'H': {'I':1, },
    'I': {'F':1, },
}

In [367]:
import copy

def graph_transpose(graph):
    out = defaultdict(dict)
    for key, value in graph.items():
        for edge, weight in value.items():
            out[edge][key] = weight
            
    return out

def graph_remove_one_node(graph, node):
    graph = copy.deepcopy(graph)
    graph.pop(node, None)
    for key, value in graph.items():
        value.pop(node, None)
    return graph

def graph_remove_a_component(graph, component):
    for node in component:
        graph = graph_remove_one_node(graph, node)
    return graph
        
def scc_algorithm(graph):
    components = []
    transposed_graph = graph_transpose(graph)
    traversal_times = depth_first_search(graph, find_min_degree(graph))
    traversal_times_decreasing = list(sort_traversal_times(traversal_times).keys())
    visited = []
    
    for node in traversal_times_decreasing:
        if node not in visited:
            # print(transposed_graph)
            this_traversal_times = depth_first_search(transposed_graph, node)
            this_component = list(this_traversal_times.keys())
            components.append(this_component)
            visited.extend(this_component)
            transposed_graph =  graph_remove_a_component(transposed_graph, this_component)

    return components

In [368]:
scc_algorithm(scc_graph)

[['A', 'E', 'B', 'D', 'G'], ['C'], ['F', 'I', 'H']]

### Prim’s Spanning Tree Algorithm 

    The solution to this problem lies in the construction of a minimum weight spanning tree. Formally we define the minimum spanning tree T for a graph G=(V,E) as follows. TTT is an acyclic subset of E that connects all the vertices in V. The sum of the weights of the edges in T is minimized.

In [369]:
from collections import defaultdict
import heapq

In [385]:
def create_spanning_tree(graph, starting_vertex):
    mst = defaultdict(set)
    visited = set([starting_vertex])
    edges = [
        (cost, starting_vertex, to)
        for to, cost in graph[starting_vertex].items()
    ]
    heapq.heapify(edges)
    
    while edges:
        cost, frm, to = heapq.heappop(edges)
        if to not in visited:
            visited.add(to)
            mst[frm].add(to)
            for to_next, cost in graph[to].items():
                if to_next not in visited:
                    heapq.heappush(edges, (cost, to, to_next))

    return mst

In [386]:
create_spanning_tree(scc_graph, "D")

defaultdict(set,
            {'D': {'B', 'G'},
             'B': {'C', 'E'},
             'C': {'F'},
             'E': {'A'},
             'F': {'H'},
             'H': {'I'}})